In [3]:
class MyClass:
    def __init__(self):
        self.id = 1234
        self.name = "Alex"
        self._smthng = "woa"

obj = MyClass()

fields = [attr for attr in dir(obj) if not attr.startswith('_')]

for field in fields:
    print(field)


id
name


In [13]:
class MyClass:
    def __init__(self):
        self.id = 1234
        self.name = "Alex"
        self._smthng = "woa"
        
    def get_name(self):
        return self.name
    
obj = MyClass()

print(obj.get_name())

my_get_name = getattr(obj, "get_name")
print(my_get_name())

Alex
Alex


In [14]:
class A:
    pass

class B(A):
    pass

class C(B):
    pass


In [16]:
def get_inheritance(cls):
    return ' -> '.join([c.__name__ for c in cls.__mro__])
print(get_inheritance(OSError))

OSError -> Exception -> BaseException -> object


In [11]:
class Tag:
    def __init__(self, name):
        self.name = name
        self.children = []

    def __enter__(self):
        return self

    def __call__(self, *args, **kwargs):
        for arg in args:
            self.children.append(str(arg))
        for key, value in kwargs.items():
            self.attributes[key] = value
        return self

    def render(self, depth=0):
        indent = '    ' * depth
        result = [f'{indent}<{self.name}']
        result.append('>\n')
        for child in self.children:
            if isinstance(child, Tag):
                result.append(child.render(depth + 1))
            else:
                result.append(f'{indent}    {child}\n')
        result.append(f'{indent}</{self.name}>\n')
        return ''.join(result)


class HTML:
    def __init__(self):
        self.code = []

    def __getattr__(self, name):
        return Tag(name)

    def get_code(self):
        return ''.join(self.code)

    def __enter__(self):
        return self


html = HTML()
with html.body():
    with html.div():
        with html.div():
            html.p('Первая строка.')
            html.p('Вторая строка.')
        with html.div():
            html.p('Третья строка.')

print(html.get_code())


CALL
CALL
CALL
CALL
CALL
CALL
CALL



In [17]:
class SVG:
    def __init__(self):
        self.lines = []
        self.circles = []

    def line(self, x1, y1, x2, y2, color='black'):
        self.lines.append((x1, y1, x2, y2, color))

    def circle(self, cx, cy, r, color='black'):
        self.circles.append((cx, cy, r, color))

    def save(self, filename, width, height):
        svg_file = open(filename, 'w')
        svg_file.write(f'<svg version="1.1" width="{width:.6f}" height="{height:.6f}" xmlns="http://www.w3.org/2000/svg">\n')
        for x1, y1, x2, y2, color in self.lines:
            svg_file.write(f'<line x1="{x1:.6f}" y1="{y1:.6f}" x2="{x2:.6f}" y2="{y2:.6f}" stroke="{color}" />\n')
        for cx, cy, r, color in self.circles:
            svg_file.write(f'<circle cx="{cx:.6f}" cy="{cy:.6f}" r="{r:.6f}" fill="{color}" />\n')
        svg_file.write('</svg>')
        svg_file.close()

svg = SVG()

svg.line(10, 10, 60, 10, color='black')
svg.line(60, 10, 60, 60, color='black')
svg.line(60, 60, 10, 60, color='black')
svg.line(10, 60, 10, 10, color='black')

svg.circle(10, 10, r=5, color='red')
svg.circle(60, 10, r=5, color='green')
svg.circle(60, 60, r=5, color='red')
svg.circle(10, 60, r=5, color='blue')

svg.save('pic.svg', 100, 100)

In [20]:
scale_x = 25
scale_y = 50

class Tree:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.x = 0
        self.y = 0

    def layout(self, x=0, y=0):
        self.x = x
        self.y = y
        if self.left:
            self.left.layout(x - scale_x, y + scale_y)
        if self.right:
            self.right.layout(x + scale_x, y + scale_y)

    def draw(self, svg):
        if self.left:
            svg.line(self.x, self.y, self.left.x, self.left.y, color='black')
            self.left.draw(svg)
        if self.right:
            svg.line(self.x, self.y, self.right.x, self.right.y, color='black')
            self.right.draw(svg)
        svg.circle(self.x, self.y, r=4, color='black')

    def save(self, filename):
        self.layout()
        svg = SVG()
        self.draw(svg)
        svg.save(filename, self.x + 20, self.y + 20)

tree_2 = Tree(2, Tree(3, Tree(4), Tree(5)), Tree(6, Tree(7)))
tree_8 = Tree(8, Tree(9, Tree(10), Tree(11, Tree(12), Tree(13))), Tree(14))
tree = Tree(1, tree_2, tree_8)
tree.save('tree.svg')
